In [1]:
import re
import os
import json
import statistics
import datetime
import warnings
from src.najdi_rok import najdi_rok
import polars as pl
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
df = pl.read_parquet(os.path.join("data","cnb_vyber.parquet"))
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))

In [3]:
preklady = df.explode("041_h").drop_nulls(subset=["041_h","rok"]).filter(pl.col("041_h") != "cze")

In [4]:
preklady_celkem = preklady.group_by(['rok']).len().rename({'len':'prekladu_celkem'})
preklady_celkem

rok,prekladu_celkem
i64,u32
1980,822
1968,727
1974,924
1971,781
1977,975
…,…
1953,1612
1947,840
1956,955


In [5]:
nahradit = {
    'eng': 'angličtina', 'ger': 'němčina', 'slo': 'slovenština', 'rus': 'ruština', 'fre': 'francouzština', 'lat': 'latina', 'pol': 'polština','ita':'italština'
}

In [11]:
nejzastoupenejsi_jazyky = preklady.group_by('041_h').len().sort('len', descending=True)
nejzastoupenejsi_jazyky = nejzastoupenejsi_jazyky.get_column('041_h').to_list()
nejzastoupenejsi_jazyky = nejzastoupenejsi_jazyky[0:7]
print(nejzastoupenejsi_jazyky)
nejzastoupenejsi_jazyky = ['eng','ger','rus','fre','slo']
preklady_po_jazycich = preklady.filter(pl.col("rok").is_between(1900,2020)).group_by(['rok','041_h']).len().rename({'len':'prekladu'})
preklady_po_jazycich = preklady_po_jazycich.join(preklady_celkem, on='rok', how='full')
preklady_po_jazycich = preklady_po_jazycich.with_columns((pl.col('prekladu') / pl.col('prekladu_celkem')).alias('podíl'))
preklady_po_jazycich = preklady_po_jazycich.with_columns(pl.col('041_h').cast(str))
preklady_po_jazycich = preklady_po_jazycich.filter(pl.col("041_h").is_in(nejzastoupenejsi_jazyky)).sort('rok').rename({'041_h':'jazyk'}).with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
preklady_po_jazycich = preklady_po_jazycich.with_columns(pl.col('jazyk').replace_strict(nahradit))
preklady_po_jazycich

['eng', 'ger', 'rus', 'fre', 'pol', 'slo', 'ita']


rok,jazyk,prekladu,rok_right,prekladu_celkem,podíl
datetime[μs],str,u32,i64,u32,f64
1900-01-01 00:00:00,"""francouzština""",51,1900,228,0.223684
1900-01-01 00:00:00,"""ruština""",23,1900,228,0.100877
1900-01-01 00:00:00,"""slovenština""",1,1900,228,0.004386
1900-01-01 00:00:00,"""angličtina""",49,1900,228,0.214912
1900-01-01 00:00:00,"""němčina""",40,1900,228,0.175439
…,…,…,…,…,…
2020-01-01 00:00:00,"""francouzština""",315,2020,4425,0.071186
2020-01-01 00:00:00,"""ruština""",64,2020,4425,0.014463
2020-01-01 00:00:00,"""angličtina""",2470,2020,4425,0.558192


In [13]:
def irozhlas_conf():

    return {
        "config" : {
             "title": {'font': "Noticia Text",
                      'fontSize': 14,
                      'anchor': 'start',
                       'fontWeight': 'bolder',
                    'subtitleFont': 'Noticia Text',
                      'subtitleFontSize': 12,
                       'subtitleFontWeight': 'lighter',
                      'subtitlePadding': 12,
                       'dy': -12,
                      },
             "axis": {
                "labelFont": "Asap",
                "titleFont": "Asap",
                 "fontWeight": "lighter",
                "titleFontWeight": "lighter",
                "labelFontSize": 10,
                "titleFontSize": 10,
                'labelPadding': 2,
                 'titlePadding': 10
            },
            "legend": {
                "labelFont": "Asap",
                "labelFontWeight": "normal",
                "titleFont": "Asap",
                "titleFontWeight": "normal",
                "labelFontSize": 10,
                "titleFontSize": 10
            }
        }
    }

alt.themes.register('irozhlas', irozhlas_conf)
alt.themes.enable('irozhlas')

ThemeRegistry.enable('irozhlas')

In [14]:
titulek = "Ze kterých jazyků se kdy překládaly knihy do češtiny"
podtitulek = ["Podíl jazyků na překladové literatuře – krásné, naučné i odborné."] #,"Angličtina a francouzština (téměř) zmizely jen během 2. světové války,","kdy dominovaly překlady z němčiny. Po osvobození raketově narostl podíl ruštiny."]
kredit = "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025"

In [15]:
podily_faceted = alt.Chart(preklady_po_jazycich.to_pandas(), title={'text': titulek}, width=300).mark_area().encode(
    alt.X("rok:T", 
          title=None,
          axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')
    ),
    alt.Y('podíl:Q', 
          axis=alt.Axis(labelExpr="datum.label * 100 + ' %'", orient='right', domainOpacity=0, tickColor='#DCDDD6'), 
          title=None
    ),
    alt.Color("jazyk:N", 
              scale=alt.Scale(range=['#687fa9','#84c0e4','#e3d83b','#b45058','#789256','#E0DAB5',]), ## '# ['#a7ddd6','#001f80','#e6e6e4','#e5db00','#e49996','#9cc002']), 
              title=None, 
              legend=None
    ),
    row=alt.Row(
        "jazyk:N", 
        title=None, 
        spacing=15, 
        header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='middle', labelFont='Asap')
    )
).resolve_axis(
    x='independent'
).properties(height=60, width=300).configure_view(stroke='transparent')

podily_faceted

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)